Author: Guy, Michael, Joshua
Summary: This is a comprenhensive simulation engine for ambulance simulation. It uses the solution from two-stage stochastic program and robust programs to simulate how ambulances would perform. The simulation mostly runs using a priority-queue of events with respect to time. Events include things like ambulance arrives to the scene of call, ambulance arrives to hospital, ect.

In [1]:
import Pkg #this line doesn't seem to run either way for me
Pkg.add("Pkg")
Pkg.add("Dates")

   Updating registry at `C:\Users\Owner\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


In [2]:
"""This activates packages, namely ambulance deployment"""

guypath = "/Users/guyfarmer/ambulance_connect/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/"
michaelpath= "/home/mhilborn/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy"
joshpath = "C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/"

currentpath = joshpath

cd(currentpath)
Pkg.activate(".")

using DataFrames, JLD, Distributions, CSV, Random, Plots,JSON, DelimitedFiles
import DataStructures: PriorityQueue, enqueue!, dequeue!
include(".//src//problem.jl")
include(".//src//model.jl")

using AmbulanceDeployment, Dates

 Activating environment at `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
┌ Info: Precompiling AmbulanceDeployment [d2c08a37-75f3-4934-bcbc-64feba624d07]
└ @ Base loading.jl:1278
┌ Warning: Package AmbulanceDeployment does not have Random in its dependencies:
│ - If you have AmbulanceDeployment checked out for development and have
│   added Random as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with AmbulanceDeployment
└ Loading Random into AmbulanceDeployment from project dependency, future warnings for AmbulanceDeployment are suppressed.


In [3]:
"""Helper functions for the simuation engine?"""
#=
Author : Ng Yeesian
Modified : Guy Farmer / Zander Tedjo / Will Worthington / Michael Hilborn
generates the closest dispatch model
Summary :
Input :
Output :
=#
mutable struct ClosestDispatch <: DispatchModel
    drivetime::DataFrame
    candidates::Vector{Vector{Int}}
    available::Vector{Int}
    assignment::Vector{Int} # which location the ambulance is assigned to
    ambulances::Vector{Vector{Int}} # list of ambulances assigned to each location
    status::Vector{Symbol} # the current status of the ambulance
        # possible statuses: :available, :responding, :atscene, :conveying, :returning
    fromtime::Vector{Int} # the time it started the new status
    hospital::Vector{Int}
end

#=
Summary :
Input :
Output : ClosestDispatch struct
=#
function ClosestDispatch(p::DeploymentProblem, drivetime::DataFrame,distribution::Vector{Int})
    candidates = Any[]
    I = 1:p.nlocations
    for region in 1:p.nregions
        push!(candidates, I[vec(p.coverage[region,:])])
    end
    #println("creating the dispatch problem :$distribution")

    assignment = zeros(Int, p.nambulances)
    ambulances = [Int[] for i in 1:p.nlocations]
    k = 1
    for i in eachindex(distribution), j in 1:distribution[i]
        assignment[k] = i
        push!(ambulances[i], k)
        k += 1
    end
    status = fill(:available, p.nambulances)
    fromtime = zeros(Int, p.nambulances)
    hospital = zeros(Int, p.nambulances)
    
    ClosestDispatch(drivetime, candidates, distribution, assignment, ambulances, status, fromtime, hospital)

end

function ClosestDispatch(p::DeploymentProblem, problem::DispatchProblem)
    stn_names = [Symbol("stn$(i)_min") for i in 1:size(p.coverage,2)]
    ClosestDispatch(p, problem.emergency_calls[:, stn_names], problem.available)
end

function update_ambulances!(model::ClosestDispatch, i::Int, delta::Int)
    model.available[i] += delta
    @assert model.available[i] >= 0
end

function closest_available(dispatch::ClosestDispatch, id::Int, problem::DispatchProblem)
    location = 0
    min_time = Inf
    for i in dispatch.candidates[problem.emergency_calls[id, :neighborhood]]
        if problem.available[i] > 0 && dispatch.drivetime[id, i] < min_time
            #println("apparently this station is available : $i and here is the amount of ambulances avalable: $(problem.available[i])")
            location = i
            min_time = dispatch.drivetime[id, i]
        end
    end
    location
end

function respond_to!(dispatch::ClosestDispatch, i::Int, t::Int)
    @assert length(dispatch.ambulances[i]) > 0 "$i: $(dispatch.ambulances[i])"
    amb = popfirst!(dispatch.ambulances[i])
    # @assert redeploy.hospital[amb] == 0
    @assert amb != 0
    @assert dispatch.status[amb] == :available "$amb: $(dispatch.status[amb])"
    dispatch.status[amb] = :responding
    #dispatch.fromtime[amb] = t
    amb
end

function arriveatscene!(dispatch::ClosestDispatch, amb::Int, t::Int)
    @assert dispatch.status[amb] == :responding "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] == 0
    dispatch.status[amb] = :atscene
    dispatch.fromtime[amb] = t
end

function arriveathospital!(dispatch::DispatchModel, amb::Int, hosp::Int, t::Int)
    @assert dispatch.status[amb] == :atscene "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :atHospital
    dispatch.fromtime[amb] = t
    dispatch.hospital[amb] = hosp
end

function returning_to!(dispatch::DispatchModel, amb::Int, t::Int)
    @assert dispatch.status[amb] == :atHospital "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :returning
    dispatch.fromtime[amb] = t
    dispatch.assignment[amb]
end

returning_to! (generic function with 1 method)

In [4]:
""" simulation parameters """
turnaround = Distributions.LogNormal(3.65, 0.3)
lambda = 0
local_path = ""
model_name = "Stochastic"
namb = 40
ncalls = 1000
#using AmbulanceDeployment

pwd()

cd(currentpath * "/test/")
pwd()

#using AmbulanceDeployment
using Dates

#include("..//src//dispatch/closestdispatch.jl")
include("..//src//problem.jl")
#include("..//src//evaluate.jl") #if you include, you need to include JuMP
#include("..//src//simulate.jl")

hospitals = CSV.File(string(local_path,"../test/austin-data/hospitals.csv")) |> DataFrame
stations = CSV.File(string(local_path,"../test/austin-data/stations.csv")) |> DataFrame
hourly_calls = CSV.File("../test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
adjacent_nbhd = CSV.File("../test/austin-data/adjacent_nbhd.csv") |> DataFrame
coverage = CSV.read("../test/austin-data/coverage_real.csv", DataFrame, header=false)
coverage = convert(Array{Bool, 2}, coverage[:, :])
incidents = CSV.File("../test/austin-data/austin_incidents.csv") |> DataFrame
solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
amb_deployment = solverstats["amb_deployment"]
model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
          "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


p = DeploymentProblem(
          hourly_calls,
          adjacent_nbhd,
          coverage,
          namb = namb,
          train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
          )
test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
#update 1: remove negative indices
test_calls = test_calls[1:ncalls,:]
println("running $namb ambuances & $ncalls calls")

# distribution of ambulances across all of the stations
x = amb_deployment[model_dict[model_name]][namb]

#establishes the dispatch problem
problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

#establishes the dispatch model
dispatch = ClosestDispatch(p, problem)

Random.seed!(1234)



running 40 ambuances & 1000 calls


MersenneTwister(UInt32[0x000004d2], Random.DSFMT.DSFMT_state(Int32[-1393240018, 1073611148, 45497681, 1072875908, 436273599, 1073674613, -2043716458, 1073445557, -254908435, 1072827086  …  -599655111, 1073144102, 367655457, 1072985259, -1278750689, 1018350124, -597141475, 249849711, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000

In [5]:
"""Simulation Enginer"""

Pkg.add("Query")
using Query

struct EMSEngine{T}
    eventlog::DataFrame
    eventqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    shortqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    guiArray::Array{Any, 1}
end

struct gui_event
    #call made / call responded / ambulance arrived / shortfall
    event_type::String
    #neighborhood where call is made from
    neighborhood_id::Int
    # station where call is made from
    deployment_id::Int
    # id of the event generated
    event_id::Int
    ambulance_id::Int
    arrival_time::Int
    remaining_amb::Int
    # current timestamp of event
    timestamp::DateTime
end

## generates the priority queue and instantiates the EMSEngine struct
function generateCalls(problem::DispatchProblem)
    ncalls = nrow(problem.emergency_calls)
    eventlog = DataFrame(
        id = 1:ncalls,
        dispatch_from = zeros(Int, ncalls),
        waittime = fill(0.0, ncalls),
        responsetime = fill(Inf, ncalls),
        scenetime = fill(Inf, ncalls),
        conveytime = fill(Inf, ncalls),
        returntime = fill(Inf, ncalls),
        return_to = zeros(Int, ncalls),
        return_type = fill(:station, ncalls),
        hospital = zeros(Int, ncalls),
        ambulance = zeros(Int, ncalls)
    )
    eventqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    shortqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    guiArray = Any[]
    for i in 1:nrow(problem.emergency_calls)
        t = problem.emergency_calls[i, :arrival_seconds]
        enqueue!(eventqueue, (:call, i, t, problem.emergency_calls[i, :neighborhood]), t)
    end
    EMSEngine{Tuple{Symbol,Int,Int,Int}}(eventlog, eventqueue, shortqueue, guiArray) ## returns the ems engine struct
end

function simulate_events!(problem::DispatchProblem, dispatch::ClosestDispatch)
    engine = generateCalls(problem)
    while(~isempty(engine.eventqueue))
        (event, id, t, value) = dequeue!(engine.eventqueue)

        if event == :call
            # if the event is reachable then change event to station2call
            # if not it is a shortfall
            println("calling event id: $id time: $t value: $value")
            call_event!(engine, problem, dispatch, id, t, value)
           
        elseif event == :station2call
            #once the ambulance arrives at the station change event to call2hospital
            println("Ambulance has arrived on the scene for event $id at time $t")
            station2call_event!(engine, problem, dispatch, id, t, value)
        elseif event == :call2hospital
            #once the ambulance arrives at the hospital it will hand over the patient and then 
            #change the deploy back to its station
            println("Ambulance for event $id has arrived at the hospital at time $t")
            call2hosp_event!(engine, problem, dispatch, id, t, value)

        elseif event == :hospital2station
            #once the ambulance drops off the patient it will reroute to the station
            println("Ambulance for event $id is returning to the station at $t")
            return_event!(engine, problem, dispatch, id, t, value)
            
        elseif event == :done
            #increase the number of ambulances at that station now that it is done
            done_event!(engine, problem, dispatch, id, t, value)
            
        end
    end
    println("This is the Short Queue")
    while(~isempty(engine.shortqueue))
       (event, id, t, value) = dequeue!(engine.shortqueue)
        print("$event, $id, $t, $value")
        print("This is not empty")
    end
    engine.eventlog , engine.guiArray
end


function call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        nbhd::Int; # the neighborhood the call is from
    )

    #check if there is an ambulance within the coverage matrix
    if sum(problem.deployment[problem.coverage[nbhd,:]]) == 0
        @assert false "$id: no ambulance reachable for call at $nbhd"
    #check if one of the ambulances within the coverage is available
    elseif sum(problem.available[problem.coverage[nbhd,:]]) > 0
        i = closest_available(dispatch, id, problem)
        #println("closest available station : $i")
        @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
        update_ambulances!(dispatch, i, -1)
        ems.eventlog[id, :dispatch_from] = i
        @assert problem.available[i] >= 0
        

        travel_time = ceil(Int, 60*problem.emergency_calls[id, Symbol("stn$(i)_min")])
        @assert travel_time >= 0
        if(t != problem.emergency_calls[id, :arrival_seconds])
            print("This is a shortfall")
        end
        ems.eventlog[id, :responsetime] = (travel_time + t - problem.emergency_calls[id,:arrival_seconds]) / 60 # minutes

        amb = respond_to!(dispatch, i, t)
        ems.eventlog[id, :ambulance] = amb

        event = gui_event("call responded", problem.emergency_calls[id, :neighborhood], i, id, amb,-1, problem.available[i] ,Dates.now())
        num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
        for i in 1:size(problem.available, 1)
            num_ambulances_array[i] = problem.available[i]
        end
        push!(ems.guiArray,event)
        push!(ems.guiArray,num_ambulances_array)
        enqueue!(ems.eventqueue, (:station2call, id, t + travel_time, amb), t + travel_time)
    #else queue it
    else
        # println(id, ": call from ", nbhd, " queued behind ", problem.wait_queue[nbhd])
        # event = gui_event("call made", nbhd, -1, id, -1,-1,-1,Dates.now())
        # push!(ems.guiArray,event)
        # problem.shortfalls = problem.shortfalls + 1
        # #push!(problem.shortfalls) # count shortfalls
        # push!(problem.wait_queue[nbhd], id) # queue the emergency call
        event = gui_event("shortfall", problem.emergency_calls[id, :neighborhood], -1, id, -1,-1,-1 ,Dates.now())
        push!(ems.guiArray, event)
        enqueue!(ems.shortqueue, (:call, id, t, problem.emergency_calls[id, :neighborhood]), t)
        println(id, ": call from ", nbhd, " is a shortfall")
    end
end

function station2call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    arriveatscene!(dispatch, amb, t)
    # time the ambulance spends at the scene
    scene_time = ceil(Int,60*0.4*rand(problem.turnaround)) # 60sec*0.4*mean(40minutes) ~ 15minutes
    ems.eventlog[id, :scenetime] = scene_time / 60 # minutes
    @assert scene_time > 0
    
    event = gui_event("ambulance arrived", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    enqueue!(ems.eventqueue, (:call2hospital, id, t + scene_time, amb), t + scene_time)
end

function call2hosp_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    h = let mintime = Inf, minindex = 0
        for h in 1:nrow(problem.hospitals)
            traveltime = problem.emergency_calls[id, Symbol("hosp$(h)_min")]
            if !isna(traveltime) && !(traveltime=="NA") && traveltime < mintime
                @assert traveltime >= 0
                minindex = h; mintime = traveltime
            end
        end
        minindex
    end
    @assert h != 0
    dispatch.hospital[amb] = ems.eventlog[id, :hospital] = h
    conveytime = 60*15 + ceil(Int, 60*problem.emergency_calls[id, Symbol("hosp$(h)_min")]) # ~20minutes
    ems.eventlog[id, :conveytime] = conveytime / 60 # minutes
    @assert conveytime >= 0 conveytime
    arriveathospital!(dispatch, amb, h, t)
    
    #         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    event = gui_event("ambulance hospital", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:hospital2station, id, t+conveytime, amb), t+conveytime)
end

function return_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    stn = returning_to!(dispatch, amb, t)
    h = dispatch.hospital[amb]
    returntime = ceil(Int,60*problem.hospitals[h, Symbol("stn$(stn)_min")]) # ~ 10minutes
    dispatch.hospital[amb] = 0
    ems.eventlog[id, :returntime] = returntime / 60 # minutes
    @assert returntime >= 0 returntime
    t_end = t + returntime
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returning", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:done, id, t_end, amb), t_end)
    # num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
    # for i in 1:size(problem.available, 1)
    #     num_ambulances_array[i] = problem.available[i]
    # end
    # push!(ems.guiArray,num_ambulances_array)
end

function done_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    i = dispatch.assignment[amb]
    println("Ambulance from event $id has returned to station $i")
    @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
    update_ambulances!(dispatch, i, 1)
    push!(dispatch.ambulances[i], amb)
    dispatch.status[amb] = :available
    ems.eventlog[id, :return_to] = id
    @assert problem.available[i] > 0
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returned", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    while(~isempty(ems.shortqueue))
        # record time difference
        (event, short_id, short_t, value) = dequeue!(ems.shortqueue)
        enqueue!(ems.eventqueue, (event, short_id, t , value), short_t)
    end
end


  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


done_event! (generic function with 1 method)

In [53]:
"""call simulation"""
#@time df, guiArray = simulate_events!(problem, dispatch)
df, guiArray = simulate_events!(problem, dispatch)

calling event id: 1 time: 38 value: 83
Ambulance has arrived on the scene for event 1 at time 338
calling event id: 2 time: 575 value: 38
calling event id: 3 time: 673 value: 129
calling event id: 4 time: 691 value: 107
Ambulance has arrived on the scene for event 3 at time 973
calling event id: 5 time: 1166 value: 113
calling event id: 6 time: 1187 value: 51
Ambulance has arrived on the scene for event 2 at time 1250
Ambulance has arrived on the scene for event 4 at time 1302
calling event id: 7 time: 1450 value: 76
Ambulance has arrived on the scene for event 5 at time 1466
Ambulance has arrived on the scene for event 6 at time 1487
Ambulance for event 1 has arrived at the hospital at time 1536
calling event id: 8 time: 1578 value: 67
calling event id: 9 time: 1640 value: 52
Ambulance for event 3 has arrived at the hospital at time 1678
calling event id: 10 time: 1838 value: 67
Ambulance has arrived on the scene for event 8 at time 1878
calling event id: 11 time: 1899 value: 67
calli

calling event id: 63 time: 19271 value: 98
Ambulance has arrived on the scene for event 62 at time 19313
Ambulance for event 61 has arrived at the hospital at time 19407
calling event id: 64 time: 19691 value: 51
Ambulance for event 55 is returning to the station at 19703
Ambulance has arrived on the scene for event 63 at time 19877
Ambulance from event 55 has returned to station 17
calling event id: 65 time: 19963 value: 98
Ambulance has arrived on the scene for event 64 at time 19991
Ambulance for event 62 has arrived at the hospital at time 20000
Ambulance for event 60 is returning to the station at 20039
Ambulance for event 59 is returning to the station at 20315
calling event id: 66 time: 20471 value: 113
Ambulance from event 60 has returned to station 11
Ambulance has arrived on the scene for event 65 at time 20686
Ambulance for event 64 has arrived at the hospital at time 20781
calling event id: 67 time: 20814 value: 36
Ambulance from event 59 has returned to station 1
Ambulance

Ambulance for event 87 is returning to the station at 31257
Ambulance for event 88 is returning to the station at 31293
Ambulance for event 91 has arrived at the hospital at time 31434
Ambulance for event 92 has arrived at the hospital at time 31487
Ambulance from event 88 has returned to station 8
Ambulance for event 89 is returning to the station at 31738
Ambulance from event 87 has returned to station 2
Ambulance from event 85 has returned to station 22
Ambulance from event 89 has returned to station 25
calling event id: 93 time: 32356 value: 66
Ambulance for event 91 is returning to the station at 32634
Ambulance for event 92 is returning to the station at 32687
Ambulance from event 91 has returned to station 8
calling event id: 94 time: 32957 value: 66
Ambulance for event 90 is returning to the station at 32965
Ambulance has arrived on the scene for event 93 at time 33076
Ambulance from event 92 has returned to station 14
Ambulance from event 90 has returned to station 39
Ambulanc

calling event id: 154 time: 53031 value: 83
Ambulance for event 141 is returning to the station at 53077
Ambulance has arrived on the scene for event 152 at time 53136
Ambulance for event 149 has arrived at the hospital at time 53153
calling event id: 155 time: 53166 value: 128
Ambulance has arrived on the scene for event 153 at time 53187
Ambulance for event 142 is returning to the station at 53190
calling event id: 156 time: 53203 value: 98
Ambulance from event 139 has returned to station 19
calling event id: 157 time: 53232 value: 65
Ambulance from event 134 has returned to station 8
Ambulance for event 150 has arrived at the hospital at time 53314
calling event id: 158 time: 53375 value: 128
Ambulance for event 143 is returning to the station at 53430
Ambulance from event 137 has returned to station 14
Ambulance from event 141 has returned to station 3
Ambulance has arrived on the scene for event 155 at time 53466
calling event id: 159 time: 53522 value: 98
Ambulance has arrived on

Ambulance has arrived on the scene for event 181 at time 59100
Ambulance has arrived on the scene for event 180 at time 59108
Ambulance for event 172 has arrived at the hospital at time 59171
calling event id: 185 time: 59176 value: 143
Ambulance has arrived on the scene for event 183 at time 59199
calling event id: 186 time: 59346 value: 97
Ambulance for event 177 has arrived at the hospital at time 59474
Ambulance has arrived on the scene for event 184 at time 59529
Ambulance from event 168 has returned to station 44
Ambulance for event 174 has arrived at the hospital at time 59686
Ambulance has arrived on the scene for event 185 at time 59732
Ambulance for event 182 has arrived at the hospital at time 59733
Ambulance for event 179 has arrived at the hospital at time 59763
Ambulance for event 178 has arrived at the hospital at time 59829
calling event id: 187 time: 59893 value: 112
Ambulance for event 180 has arrived at the hospital at time 59925
Ambulance has arrived on the scene fo

...

Ambulance for event 866 has arrived at the hospital at time 246485
Ambulance for event 851 is returning to the station at 246512
Ambulance for event 864 has arrived at the hospital at time 246558
Ambulance from event 854 has returned to station 11
Ambulance for event 868 has arrived at the hospital at time 246615
Ambulance from event 852 has returned to station 1
Ambulance for event 858 has arrived at the hospital at time 246674
Ambulance for event 856 is returning to the station at 246702
Ambulance from event 850 has returned to station 2
Ambulance has arrived on the scene for event 871 at time 246768
Ambulance from event 855 has returned to station 26
Ambulance for event 859 is returning to the station at 246844
Ambulance from event 857 has returned to station 28
Ambulance has arrived on the scene for event 870 at time 247062
Ambulance from event 856 has returned to station 8
Ambulance for event 861 is returning to the station at 247149
Ambulance for event 869 has arrived at the hosp

calling event id: 898 time: 252637 value: 129
Ambulance for event 891 has arrived at the hospital at time 252673
Ambulance for event 893 has arrived at the hospital at time 252690
Ambulance has arrived on the scene for event 894 at time 252760
calling event id: 899 time: 252799 value: 68
calling event id: 900 time: 252837 value: 71
Ambulance for event 889 is returning to the station at 252846
calling event id: 901 time: 252877 value: 129
Ambulance from event 886 has returned to station 28
Ambulance has arrived on the scene for event 897 at time 252920
Ambulance has arrived on the scene for event 898 at time 252937
Ambulance for event 884 is returning to the station at 253018
Ambulance has arrived on the scene for event 899 at time 253099
calling event id: 902 time: 253137 value: 67
Ambulance from event 884 has returned to station 8
calling event id: 903 time: 253176 value: 83
calling event id: 904 time: 253220 value: 82
Ambulance has arrived on the scene for event 896 at time 253288
Am

calling event id: 928 time: 259399 value: 82
Ambulance has arrived on the scene for event 923 at time 259492
Ambulance for event 918 is returning to the station at 259509
Ambulance has arrived on the scene for event 927 at time 259509
calling event id: 929 time: 259529 value: 116
Ambulance from event 910 has returned to station 2
Ambulance has arrived on the scene for event 925 at time 259575
Ambulance for event 919 has arrived at the hospital at time 259635
Ambulance from event 917 has returned to station 19
Ambulance has arrived on the scene for event 928 at time 259699
Ambulance from event 913 has returned to station 15
Ambulance for event 921 has arrived at the hospital at time 259771
Ambulance has arrived on the scene for event 929 at time 259829
Ambulance from event 911 has returned to station 2
Ambulance has arrived on the scene for event 926 at time 259884
calling event id: 930 time: 259902 value: 113
Ambulance from event 918 has returned to station 11
Ambulance from event 900 

Ambulance for event 937 is returning to the station at 264830
Ambulance for event 941 is returning to the station at 264963
Ambulance for event 948 is returning to the station at 264978
Ambulance for event 945 is returning to the station at 264979
Ambulance from event 939 has returned to station 28
Ambulance from event 940 has returned to station 1
Ambulance from event 943 has returned to station 15
Ambulance from event 948 has returned to station 8
Ambulance for event 952 has arrived at the hospital at time 265172
Ambulance for event 942 is returning to the station at 265271
calling event id: 958 time: 265275 value: 51
Ambulance from event 937 has returned to station 25
Ambulance for event 944 is returning to the station at 265300
Ambulance from event 941 has returned to station 11
Ambulance for event 946 is returning to the station at 265454
Ambulance has arrived on the scene for event 955 at time 265458
Ambulance has arrived on the scene for event 957 at time 265490
Ambulance from e

(1000×11 DataFrame. Omitted printing of 6 columns
│ Row  │ id    │ dispatch_from │ waittime │ responsetime │ scenetime │
│      │ Int64 │ Int64         │ Float64  │ Float64      │ Float64   │
├──────┼───────┼───────────────┼──────────┼──────────────┼───────────┤
│ 1    │ 1     │ 1             │ 0.0      │ 5.0          │ 19.9667   │
│ 2    │ 2     │ 43            │ 0.0      │ 11.25        │ 13.2833   │
│ 3    │ 3     │ 36            │ 0.0      │ 5.0          │ 11.75     │
│ 4    │ 4     │ 34            │ 0.0      │ 10.1833      │ 11.75     │
│ 5    │ 5     │ 14            │ 0.0      │ 5.0          │ 19.95     │
│ 6    │ 6     │ 22            │ 0.0      │ 5.0          │ 29.8833   │
│ 7    │ 7     │ 39            │ 0.0      │ 24.55        │ 26.8333   │
│ 8    │ 8     │ 8             │ 0.0      │ 5.0          │ 18.0667   │
│ 9    │ 9     │ 8             │ 0.0      │ 11.35        │ 13.1833   │
│ 10   │ 10    │ 8             │ 0.0      │ 5.0          │ 14.2      │
⋮
│ 990  │ 990   │ 44      

In [6]:
"""batch call simulation"""
batches = 12
test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
#update 1: remove negative indices
using DelimitedFiles

for i = 1:12
    """ simulation parameters """
    p = DeploymentProblem(
              hourly_calls,
              adjacent_nbhd,
              coverage,
              namb = namb,
              train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
              )
    test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
    #update 1: remove negative indices
    start_ind = (i-1)*1000+1
    end_ind = (i)*1000
    println("running $namb ambuances & $start_ind to $end_ind calls")
    test_calls_batch = test_calls[start_ind:end_ind,:]

    # distribution of ambulances across all of the stations
    x = amb_deployment[model_dict[model_name]][namb]

    #establishes the dispatch problem
    problem = DispatchProblem(test_calls_batch, hospitals, stations, p.coverage, x, turnaround=turnaround)

    #establishes the dispatch model
    dispatch = ClosestDispatch(p, problem)

    #@time df, guiArray = simulate_events!(problem, dispatch)
    df, guiArray = simulate_events!(problem, dispatch)
    print(mean(df["responsetime"]))
    
    filename = string(model_name,string(namb),"calls",string(ncalls),"batch",string(i),".csv")
    open(filename, "w") do filename
        writedlm(filename,  df["responsetime"])
    end
    print(filename)
end

running 40 ambuances & 1 to 1000 calls
calling event id: 1 time: 38 value: 83
Ambulance has arrived on the scene for event 1 at time 338
calling event id: 2 time: 575 value: 38
calling event id: 3 time: 673 value: 129
calling event id: 4 time: 691 value: 107
calling event id: 5 time: 1166 value: 113
calling event id: 6 time: 1187 value: 51
Ambulance has arrived on the scene for event 2 at time 1250
Ambulance has arrived on the scene for event 4 at time 1302
Ambulance has arrived on the scene for event 3 at time 1407
calling event id: 7 time: 1450 value: 76
Ambulance has arrived on the scene for event 6 at time 1487
Ambulance for event 1 has arrived at the hospital at time 1536
calling event id: 8 time: 1578 value: 67
calling event id: 9 time: 1640 value: 52
Ambulance has arrived on the scene for event 5 at time 1752
calling event id: 10 time: 1838 value: 67
Ambulance has arrived on the scene for event 8 at time 1878
calling event id: 11 time: 1899 value: 67
calling event id: 12 time: 1

Ambulance has arrived on the scene for event 36 at time 10200
Ambulance for event 31 is returning to the station at 10320
Ambulance for event 27 is returning to the station at 10427
calling event id: 38 time: 10595 value: 98
Ambulance for event 29 is returning to the station at 10609
Ambulance from event 27 has returned to station 17
Ambulance for event 26 is returning to the station at 10676
Ambulance from event 21 has returned to station 43
Ambulance has arrived on the scene for event 37 at time 10699
Ambulance for event 35 has arrived at the hospital at time 10727
calling event id: 39 time: 10727 value: 52
calling event id: 40 time: 10770 value: 98
Ambulance from event 31 has returned to station 25
Ambulance for event 30 is returning to the station at 10795
Ambulance from event 26 has returned to station 8
Ambulance for event 28 is returning to the station at 10926
Ambulance for event 34 has arrived at the hospital at time 10933
Ambulance from event 29 has returned to station 3
Ambu

Ambulance for event 55 is returning to the station at 19703
Ambulance has arrived on the scene for event 63 at time 19879
Ambulance from event 55 has returned to station 17
calling event id: 65 time: 19963 value: 98
Ambulance has arrived on the scene for event 64 at time 19991
Ambulance for event 62 has arrived at the hospital at time 20000
Ambulance for event 60 is returning to the station at 20037
Ambulance for event 59 is returning to the station at 20315
calling event id: 66 time: 20471 value: 113
Ambulance from event 60 has returned to station 14
Ambulance for event 64 has arrived at the hospital at time 20781
Ambulance has arrived on the scene for event 65 at time 20785
calling event id: 67 time: 20814 value: 36
Ambulance from event 59 has returned to station 1
calling event id: 68 time: 21018 value: 66
Ambulance has arrived on the scene for event 66 at time 21057
Ambulance for event 61 is returning to the station at 21130
Ambulance for event 62 is returning to the station at 212

Ambulance for event 87 is returning to the station at 31725
Ambulance for event 89 is returning to the station at 31738
Ambulance from event 85 has returned to station 22
Ambulance from event 89 has returned to station 25
calling event id: 93 time: 32356 value: 66
Ambulance for event 91 is returning to the station at 32634
Ambulance has arrived on the scene for event 93 at time 32656
Ambulance for event 92 is returning to the station at 32687
Ambulance from event 87 has returned to station 22
Ambulance from event 91 has returned to station 8
calling event id: 94 time: 32957 value: 66
Ambulance for event 90 is returning to the station at 32965
Ambulance from event 92 has returned to station 14
Ambulance has arrived on the scene for event 94 at time 33257
Ambulance from event 90 has returned to station 39
Ambulance for event 93 has arrived at the hospital at time 33657
calling event id: 95 time: 33904 value: 99
calling event id: 96 time: 33986 value: 142
Ambulance has arrived on the scen

Ambulance for event 118 has arrived at the hospital at time 45305
Ambulance has arrived on the scene for event 122 at time 45307
Ambulance from event 113 has returned to station 17
Ambulance from event 111 has returned to station 43
Ambulance for event 119 has arrived at the hospital at time 45372
Ambulance has arrived on the scene for event 123 at time 45401
calling event id: 126 time: 45420 value: 143
Ambulance for event 120 has arrived at the hospital at time 45779
Ambulance for event 117 has arrived at the hospital at time 45779
calling event id: 127 time: 45780 value: 97
Ambulance has arrived on the scene for event 124 at time 45854
Ambulance has arrived on the scene for event 126 at time 45976
Ambulance for event 116 is returning to the station at 46084
calling event id: 128 time: 46110 value: 82
Ambulance has arrived on the scene for event 127 at time 46154
Ambulance for event 121 has arrived at the hospital at time 46279
Ambulance has arrived on the scene for event 125 at time 

...

Ambulance from event 536 has returned to station 16
Ambulance has arrived on the scene for event 545 at time 142330
Ambulance for event 541 is returning to the station at 142553
Ambulance from event 534 has returned to station 22
calling event id: 546 time: 142676 value: 98
Ambulance for event 537 is returning to the station at 142715
calling event id: 547 time: 142762 value: 82
Ambulance for event 542 is returning to the station at 142902
Ambulance from event 541 has returned to station 18
Ambulance from event 542 has returned to station 8
Ambulance has arrived on the scene for event 547 at time 143062
Ambulance for event 545 has arrived at the hospital at time 143150
Ambulance for event 543 is returning to the station at 143216
Ambulance from event 537 has returned to station 5
Ambulance has arrived on the scene for event 546 at time 143282
Ambulance for event 540 is returning to the station at 143287
calling event id: 548 time: 143324 value: 51
calling event id: 549 time: 143378 val

calling event id: 577 time: 149388 value: 68
Ambulance for event 573 has arrived at the hospital at time 149415
calling event id: 578 time: 149448 value: 98
Ambulance has arrived on the scene for event 574 at time 149475
Ambulance for event 564 is returning to the station at 149499
Ambulance has arrived on the scene for event 576 at time 149501
Ambulance for event 571 has arrived at the hospital at time 149559
Ambulance for event 570 has arrived at the hospital at time 149585
Ambulance from event 566 has returned to station 9
Ambulance for event 565 is returning to the station at 149712
Ambulance has arrived on the scene for event 575 at time 149793
Ambulance from event 553 has returned to station 43
calling event id: 579 time: 149845 value: 113
Ambulance for event 568 is returning to the station at 149874
Ambulance has arrived on the scene for event 577 at time 149890
Ambulance from event 565 has returned to station 17
Ambulance for event 576 has arrived at the hospital at time 149988

calling event id: 601 time: 158342 value: 129
This is a shortfallAmbulance has arrived on the scene for event 599 at time 158383
Ambulance from event 591 has returned to station 19
Ambulance from event 595 has returned to station 3
Ambulance for event 598 has arrived at the hospital at time 158666
Ambulance for event 600 has arrived at the hospital at time 158799
Ambulance for event 599 has arrived at the hospital at time 158872
Ambulance from event 594 has returned to station 9
Ambulance for event 590 is returning to the station at 159109
Ambulance has arrived on the scene for event 601 at time 159110
Ambulance for event 596 has arrived at the hospital at time 159267
Ambulance for event 597 is returning to the station at 159360
Ambulance from event 597 has returned to station 5
Ambulance for event 600 is returning to the station at 159999
calling event id: 602 time: 160033 value: 98
calling event id: 603 time: 160226 value: 83
Ambulance for event 598 is returning to the station at 160

Excessive output truncated after 524296 bytes.


calling event id: 626 time: 169856 value: 111
Ambulance has arrived on the scene for event 623 at time 169899
Ambulance for event 615 is returning to the station at 169996
calling event id: 627 time: 170010 value: 52
Ambulance has arrived on the scene for event 625 at time 170040

In [7]:
print(mean(df["responsetime"]))

LoadError: [91mUndefVarError: df not defined[39m

In [93]:
array = filter(x->typeof(x)==gui_event,guiArray)
responded_array = filter(x->x.event_type == "call responded", array)

1000-element Array{Any,1}:
 gui_event("call responded", 36, 27, 1, 25, -1, 0, DateTime("2021-05-09T02:54:11.7"))
 gui_event("call responded", 128, 25, 2, 20, -1, 3, DateTime("2021-05-09T02:54:11.716"))
 gui_event("call responded", 81, 13, 3, 9, -1, 0, DateTime("2021-05-09T02:54:11.731"))
 gui_event("call responded", 82, 21, 4, 17, -1, 2, DateTime("2021-05-09T02:54:11.778"))
 gui_event("call responded", 96, 17, 5, 11, -1, 0, DateTime("2021-05-09T02:54:11.855"))
 gui_event("call responded", 51, 2, 6, 1, -1, 4, DateTime("2021-05-09T02:54:11.886"))
 gui_event("call responded", 83, 21, 7, 18, -1, 1, DateTime("2021-05-09T02:54:11.901"))
 gui_event("call responded", 83, 21, 8, 19, -1, 0, DateTime("2021-05-09T02:54:11.932"))
 gui_event("call responded", 113, 25, 9, 21, -1, 2, DateTime("2021-05-09T02:54:11.947"))
 gui_event("call responded", 99, 4, 10, 6, -1, 0, DateTime("2021-05-09T02:54:12.057"))
 gui_event("call responded", 83, 43, 11, 39, -1, 0, DateTime("2021-05-09T02:54:12.072"))
 gui_eve

In [91]:
mean(df["responsetime"])

37.61875

In [25]:
using DelimitedFiles
filename = string(model_name,string(namb),"calls",string(ncalls),".csv")
open(filename, "w") do filename
    writedlm(filename,  df["responsetime"])
end
print(filename)

Stochastic50calls1000.csv

In [18]:
#using CSV
#filename = string(joshpath,"test/",model_name,string(namb),"calls",string(ncalls),".csv")
#CSV.write(filename, df, delim=",")